# **Inteligencia de negocios - ISIS3301**

# **Proyecto 1 - Analitica de texto**

## **Sección 2**
## **Grupo 23**
*   Rafael Santiago Bastos Russi - *202110792*
*   David Santiago Valderrama Herrera - *201910987*
*   Jesús Alejandro Dávila Pinchao - *202014263*




# **Caso de estudio**

La Organización de las Naciones Unidas (ONU) adopta, el 25 de septiembre del año 2015, la
Agenda 2030i para el desarrollo sostenible, cuyo fin es reducir la pobreza, garantizar acceso
a la salud y educación, buscar igualdad de género y oportunidades, disminuir el impacto
ambiental, entre otros. Esta agenda se basa en 17 objetivos de desarrollo sostenibleii (ODS)
y 169 metas (derivadas de los diferentes ODS).
Dentro del trabajo en conjunto de diferentes entes para alcanzar el cumplimiento de los
ODS, muchas entidades tienen como enfoque el seguimiento y la evaluación de las políticas
públicas y su impacto a nivel social. Este es el caso del Fondo de Poblaciones de las Naciones
Unidas (UNFPAiii) que, junto con entidades públicas y haciendo uso de diferentes
herramientas de participación ciudadana, busca identificar problemas y evaluar soluciones
actuales, relacionando la información con los diferentes ODS. En este contexto, uno de los
procesos que requiere de un mayor esfuerzo es la clasificación de la información textual
que es recopilada, ya que es una tarea que consume gran cantidad recursos y para la cual
se requiere un experto. Es por esto que, en los últimos años, UNFPA ha venido trabajando,
en conjunto con la Universidad de los Andes, en la implementación de diferentes estrategias
de clasificación de textos, que les permitan hacer un análisis automatizado de opiniones
que representan la voz de los habitantes locales sobre problemáticas de su entorno
particular.
Para apoyar a UNFPA en este proceso se ha planteado el primer proyecto del curso, cuyo
objetivo es desarrollar un modelo de clasificación, con técnicas de aprendizaje
automático, que permita relacionar de manera automática un texto según los ODS. Al
igual que desarrollar una aplicación que facilite la interacción con el resultado de dicho
modelo. El modelo podrá ser utilizado entonces para la interpretación y análisis de la
información textual que es recopilada a través de diferentes fuentes por UNFPA en procesos
de planeación participativa para el desarrollo a nivel territorial.

#**Etapa 1**

## **1. Importación de librerias**

In [ ]:
!pip install num2words
!pip install inflect
!pip install spacy

In [ ]:
import pandas as pd
import numpy as np
import sys
import re, string, unicodedata

import inflect
import nltk
import spacy
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score

# Versiones anteriores a 1.2 de sklearn: from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
from num2words import num2words
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

#Librerías para la visualización
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns 


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
!python -m spacy download es_core_news_sm

## **2. Perfilamiento y entendimiento de los datos**

### **2.1. Lectura de los datos**

In [ ]:
data=pd.read_excel('./data/cat_345.xlsx')
# Asignación a una nueva variable de los datos leidos

### **2.2. Entendimiento de los datos**

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
data.describe()

In [ ]:
pd.value_counts(data['sdg'])

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated(keep = False).sum()

## **3. Preparación de los datos**

Se realizaran las siguientes actividades para una adecuada preparación de los datos:



*   Limpieza de los datos.
*   Tokenización.
*   Normalización.

Usando las librerias spacy para el procesamiento de las palabras incluyendo su lematización y verificación de números, unicodedata para eliminar caracteres especiales y num2words para convertir numeros en palabras


## **3.1. Limpieza y tokenización**

In [ ]:
data_t = data.copy()

In [ ]:
nlp = spacy.load('es_core_news_sm')

Se carga el modulo para español de spacy

In [ ]:
texts = data_t['Textos_espanol']
tokens = []

stop_words = nlp.Defaults.stop_words  #Stop words en español
cont = 0
for opinion in texts:
  opinionP = opinion.lower() #Se pone el texto en minusculas
  opinionP = unicodedata.normalize('NFKD', opinionP).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  #Se quitan caracteres especiales
  opinionDoc = nlp(opinionP) #Se crea un doc con npl para procesar el texto
  tokensI = []
  for word in opinionDoc:
    wordP = re.sub(r'[^\w\s]', '', word.text) #Remover signos de puntuación
    if wordP != '':
      if wordP == "15.7":
           print(wordP)
      if word.is_digit:
        #print(wordP)

        num_word = num2words(wordP, lang='es')
        #print(num_word)
        tokensI.append(num_word)
        if cont == 5:
           print(wordP)
           print(num_word)

      else:
        if word.text not in stop_words: #No se tienen en cuenta las stop words
            tokensI.append(word.lemma_) #Se toma en cuenta solo el lemma de la palabra
  cont+=1
  tokens.append(tokensI)

In [ ]:
data_t['words'] = pd.Series(tokens, copy=False)
data_t.head()

In [ ]:
data_t["words"].dtype

In [ ]:
datix = data_t["words"]

## **3.2. Normalización**

In [ ]:
data_t['words'] = data_t['words'].apply(lambda x: ' '.join(map(str, x)))
data_t

In [ ]:
data_t["words"].dtype

In [ ]:
X_data, Y_data = data_t['words'], data_t['sdg'].astype(int)

In [ ]:
tf_idf = TfidfVectorizer(max_features=3000)
X_data = tf_idf.fit_transform(X_data)

print(X_data.shape)
X_data.toarray()[0]

Para normalizar se usa un vectorizador TF-IDF, quedando la variable predictora en Y_data y los textos en X_data

# **4. Modelos de clasificación**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=0)

## **4.1. Primer modelo: Árbol de decisión**

In [ ]:
clf = DecisionTreeClassifier(random_state = 1)

### 4.1.1. **Modelo de Prueba**

In [ ]:
clf.fit(X_test, y_test)

In [ ]:
# Predecir las categorías en el conjunto de entrenamiento
y_pred_test = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_test)
print(f'Precisión: {accuracy}')

report = classification_report(y_test, y_pred_test)
print(report)

#### **Matriz de confusión**

In [ ]:
# Se genera la matriz de confusión
cm_test = confusion_matrix(y_test, y_pred_test, labels = clf.classes_)
cm_test_norm = confusion_matrix(y_test, y_pred_test, labels = clf.classes_, normalize = 'true')

In [ ]:
# Se puede visualizar la matriz de confusión
#plot_confusion_matrix(arbol, X_test, Y_test)}
disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=clf.classes_)
disp_test.plot(cmap=plt.cm.Blues)

disp_test_norm = ConfusionMatrixDisplay(confusion_matrix=cm_test_norm, display_labels=clf.classes_)
disp_test_norm.plot(cmap=plt.cm.Blues)

plt.show()

### 4.1.2. **Modelo de Entrenamiento**

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
# Predecir las categorías en el conjunto de entrenamiento
y_pred_train = clf.predict(X_train)

accuracy = accuracy_score(y_train, y_pred_train)
print(f'Precisión: {accuracy}')

report = classification_report(y_train, y_pred_train)
print(report)

#### **Matriz de Confusión**

In [ ]:
# Se genera la matriz de confusión
cm_train = confusion_matrix(y_train, y_pred_train, labels = clf.classes_)
cm_train_norm = confusion_matrix(y_train, y_pred_train, labels = clf.classes_, normalize = 'true')

In [ ]:
# Visualización de la matriz de confusión

disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=clf.classes_)
disp_train.plot(cmap=plt.cm.Blues)

disp_train_norm = ConfusionMatrixDisplay(confusion_matrix=cm_train_norm, display_labels=clf.classes_)
disp_train_norm.plot(cmap=plt.cm.Blues)

plt.show()

## **4.2. Segundo modelo: K-Nearest Neighbors (KNN)**

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors = 3)

In [ ]:
# Entrenamiento del "modelo"
knn_model.fit(X_train, y_train)

### **4.2.1. Modelo de Entrenamiento**

In [ ]:
# Generación de predicciones para prueba
preds_train = knn_model.predict(X_train)

accuracy = accuracy_score(y_train, preds_train)
print(f'Precisión: {accuracy}')

report = classification_report(y_train, preds_train)
print(report)

#### **Matriz de Confusión**

In [ ]:
# Generación de la matriz de confusión
cm_train = confusion_matrix(y_train, preds_train, labels = knn_model.classes_)
cm_train_norm = confusion_matrix(y_train, preds_train, labels = knn_model.classes_, normalize = 'true')

In [ ]:
# Visualización de la matriz
disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=knn_model.classes_)
disp_train.plot(cmap=plt.cm.Blues)

disp_train_norm = ConfusionMatrixDisplay(confusion_matrix=cm_train_norm, display_labels=knn_model.classes_)
disp_train_norm.plot(cmap=plt.cm.Blues)

plt.show()

### **4.2.1. Modelo de Prueba**

In [ ]:
# Generación de predicciones para prueba
preds_test = knn_model.predict(X_test)

accuracy = accuracy_score(y_test, preds_test)
print(f'Precisión: {accuracy}')

report = classification_report(y_test, preds_test)
print(report)

#### **Matriz de confusión**

In [ ]:
# Generación de la matriz de confusión
cm_test = confusion_matrix(y_test, preds_test, labels = knn_model.classes_)
cm_test_norm = confusion_matrix(y_test, preds_test, labels = knn_model.classes_, normalize = 'true')

In [ ]:
# Visualización de la matriz
disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=knn_model.classes_)
disp_test.plot(cmap=plt.cm.Blues)

disp_test_norm = ConfusionMatrixDisplay(confusion_matrix=cm_test_norm, display_labels=knn_model.classes_)
disp_test_norm.plot(cmap=plt.cm.Blues)

plt.show()

## **4.3. Tercer modelo: TF-IDF con Random Forest**

In [ ]:
tfidf_model = RandomForestClassifier(random_state=3)

### 4.3.1. **Modelo de Entrenamiento**

In [ ]:
tfidf_model.fit(X_train, y_train)

In [ ]:
tfidf_estimators = tfidf_model.estimators_
print("Number of trees:", len(tfidf_estimators))
print("Trees depth (mean):", np.mean([tree.get_depth() for tree in tfidf_estimators]))

In [ ]:
y_train_tfidf_predict = tfidf_model.predict(X_train)

In [ ]:
accuracy = accuracy_score(y_train, y_train_tfidf_predict)
print(f'Precisión: {accuracy}')

report = classification_report(y_train, y_train_tfidf_predict)
print(report)

#### **Matriz de Confusión**

In [ ]:
# Generación de la matriz de confusión
cm_train = confusion_matrix(y_train, y_train_tfidf_predict, labels = tfidf_model.classes_)
cm_train_norm = confusion_matrix(y_train, y_train_tfidf_predict, labels = tfidf_model.classes_, normalize = 'true')

In [ ]:
# Visualización de la matriz
disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=tfidf_model.classes_)
disp_train.plot(cmap=plt.cm.Blues)

disp_train_norm = ConfusionMatrixDisplay(confusion_matrix=cm_train_norm, display_labels=tfidf_model.classes_)
disp_train_norm.plot(cmap=plt.cm.Blues)

plt.show()

### 4.3.2. Modelo de Prueba

In [ ]:
tfidf_model.fit(X_test, y_test)

In [ ]:
tfidf_estimators = tfidf_model.estimators_
print("Number of trees:", len(tfidf_estimators))
print("Trees depth (mean):", np.mean([tree.get_depth() for tree in tfidf_estimators]))

In [ ]:
y_test_tfidf_predict = tfidf_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_test_tfidf_predict)
print(f'Precisión: {accuracy}')

report = classification_report(y_test, y_test_tfidf_predict)
print(report)

#### **Matriz de Confusión**

In [ ]:
# Generación de la matriz de confusión
cm_test= confusion_matrix(y_test, y_test_tfidf_predict, labels = tfidf_model.classes_)
cm_test_norm = confusion_matrix(y_test, y_test_tfidf_predict, labels = tfidf_model.classes_, normalize = 'true')

In [ ]:
# Visualización de la matriz
disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=tfidf_model.classes_)
disp_test.plot(cmap=plt.cm.Blues)

disp_test_norm = ConfusionMatrixDisplay(confusion_matrix=cm_test_norm, display_labels=tfidf_model.classes_)
disp_test_norm.plot(cmap=plt.cm.Blues)

plt.show()

# **Selección del modelo**

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [4, 8, 12, 16, 20, 24, 28, 32, 36, 40],
    'min_samples_split': [2, 3, 4, 5]
}

In [ ]:
grid_tree_model = GridSearchCV(clf, param_grid, scoring = ['precision', 'recall', 'f1'], refit = 'f1', cv = 10, n_jobs = -1, verbose = 1)

In [ ]:
grid_tree_model.fit(X_train, y_train)

In [ ]:
cv_results_df = pd.DataFrame(grid_tree_model.cv_results_)

In [ ]:
cv_results_df.head()

In [ ]:
grid_tree_model.best_params_

In [ ]:
grid_tree_model.best_score_

In [ ]:
unlabeled_data = pd.read_excel('./data/SinEtiquetatest_cat_345.xlsx')


In [ ]:
def tagging(data):
    data_t = data.copy()
    global nlp
    texts = data_t['Textos_espanol']
    tokens = []

    stop_words = nlp.Defaults.stop_words  #Stop words en español
    cont = 0
    for opinion in texts:
        opinionP = opinion.lower() #Se pone el texto en minusculas
        opinionP = unicodedata.normalize('NFKD', opinionP).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        #Se quitan caracteres especiales
        opinionDoc = nlp(opinionP) #Se crea un doc con npl para procesar el texto
        tokensI = []
        for word in opinionDoc:
            wordP = re.sub(r'[^\w\s]', '', word.text) #Remover signos de puntuación
            if wordP != '':
                if wordP == "15.7":
                    print(wordP)
                if word.is_digit:

                    num_word = num2words(wordP, lang='es')
                    tokensI.append(num_word)

                else:
                    if word.text not in stop_words: #No se tienen en cuenta las stop words
                        tokensI.append(word.lemma_) #Se toma en cuenta solo el lemma de la palabra
        cont+=1
        tokens.append(tokensI)
    data_t['words'] = pd.Series(tokens, copy=False)
    data_t['words'] = data_t['words'].apply(lambda x: ' '.join(map(str, x)))
    global unlabeled_X_data
    unlabeled_X_data = data_t['words']
    unlabeled_X_data = tf_idf.fit_transform(unlabeled_X_data)
    tfidf_model.predict(unlabeled_X_data)
    columns_to_drop = ['predicted_labels', 'predicted_sdg']
    data_x = data.drop(columns_to_drop, axis=1)
    data_x.to_csv('./data/labeled_unlabeled_data.csv', index=False)

In [ ]:
tagging(unlabeled_data)